In [6]:
import numpy as np
from sklearn import manifold
from dreimac import GeometryUtils, CircleMapUtils, CircularCoords
from ripser import ripser
from persim import plot_diagrams
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
penta2 = np.loadtxt('data/genus_3_clean.txt', delimiter=',')

print(penta2.shape)

(37167, 10)


In [3]:
n_land = 900
k_neigh = 20
dist_mat, pointcloud_permutation = GeometryUtils.landmark_geodesic_distance(penta2, n_land, k_neigh)

In [8]:
angle_data = np.zeros((penta2.shape[0],5)) # internal angles

for i in range(5):
    ii = (2*i -2)%10
    jj = (2*i - 1)%10
    u = penta2[:, [ii, jj]]
    
    ii = (2*i)%10
    jj = (2*i + 1)%10
    v = penta2[:, [ii , jj]]
    
    ii = (2*i + 2)%10
    jj = (2*i + 3)%10
    w = penta2[:, [ii, jj]]
    
    z = np.sum((u - v)*(w-v), axis = 1)
    
    angle_data[:,i]  = np.arccos( np.minimum(z,1) )
    
energy = np.sum((angle_data - (3/5)*np.pi)**2 , axis = 1)

In [ ]:
#implementing circular coords to find cuts found via persistent cohomology genus 3 jupyter notebook

n_lands = 900

cc = CircularCoords(dist_mat, n_landmarks= n_lands, distance_matrix = True)

coho_classes = [0, 1, 2, 3, 4, 5]

circular_coords = []

for i in coho_classes:
    circular_coords.append(cc.get_coordinates(perc = 0.6, cocycle_idx=i, standard_range= False))

#Applying Circular coords and doing TDR

thetas= CircleMapUtils.linear_combination(np.array(circular_coords), [[1,0,0,0,0,0], [0,1,0,0,0,0],[0,0,1,0,0,0], [0,0,0,1,0,0], [0,0,0,0,1,0],[0,0,0,0,0,1]])
#Standard Basis, 1,2,6 little fold

theta1 = CircleMapUtils.center(thetas[0])
theta2 = CircleMapUtils.center(thetas[1])
theta3 = CircleMapUtils.center(thetas[2])
theta4 = CircleMapUtils.center(thetas[3])
theta5 = CircleMapUtils.center(thetas[4])
theta6 = CircleMapUtils.center(thetas[5])

# fig = go.Figure(data=[go.Scatter3d(
#     x=theta1, y=theta2, z=theta6, 
#     mode ='markers', 
#     marker=dict(size = 3, color =energy[pointcloud_permutation], colorscale='plasma')
# )])

# fig.update_layout(autosize=False, width=700, height=700)

# fig.show()




In [ ]:
n_lands=1000

coho_data = np.array([theta1, theta2, theta6])

coho = CircularCoords(coho_data.T , n_landmarks= n_lands)

coho_classes = [0, 1, 2, 3, 4, 5]

circular_coords_coho = []

for i in coho_classes:
    circular_coords_coho.append(coho.get_coordinates(perc = 0.6, cocycle_idx=i, standard_range= False))
    
thetas_prime = CircleMapUtils.linear_combination(circular_coords_coho,[[1,0,0,0,0,0], [0,1,0,0,0,3],[1,0,1,0,0,0],[1,0,0,1,0,0],[0,0,0,0,2,1],[0,0,0,0,0,1]])

fig = make_subplots(rows=1, cols=len(coho_classes), 
                    subplot_titles = tuple('Circ Coord '+str(i) for i in coho_classes),
                    specs = [[{'type': 'scatter3d'} for i in coho_classes]])

for i, coord in enumerate(circular_coords_coho):
    fig.add_trace(
        go.Scatter3d(x=theta1, y=theta2, z=theta6,   mode ='markers', 
        marker=dict(size = 2 , color = (thetas_prime[i]))),  
        row=1, col=i+1)
    fig.update_scenes(xaxis=dict( ticks='', showticklabels=False), 
                      yaxis=dict( ticks='', showticklabels=False), 
                      zaxis=dict( ticks='', showticklabels=False), 
                      aspectmode='data', row=1, col=i+1)

fig.update_layout( showlegend=False )
fig.show()

In [ ]:
mds= manifold.MDS(
    n_components=2,
    metric=True,
    dissimilarity="precomputed",
)
mds.fit(dist_mat)